In [1]:
import os
import numpy as np
from netCDF4 import Dataset
import pandas as pd
from collections import defaultdict
from calendar import monthrange


In [2]:

# Assuming files are named like 'data_YYYYMMDD.nc'
data_dir = '../results/daily/gcomc/chla'
file_pattern = 'data_{}.nc'  # Placeholder for file name format
lat_min_crop, lat_max_crop = 34.4, 35.7
lon_min_crop, lon_max_crop = 138.35, 140.2

In [3]:
land_mask_accumulator = None
count = 0

# First pass to identify land pixels
for file_name in os.listdir(data_dir):
    if not file_name.endswith('.nc'):
       continue
    
    count += 1
    if count % 100 == 0:
        print(file_name)

    if file_name.endswith('.nc'):
        with Dataset(os.path.join(data_dir, file_name), 'r') as nc:
            lat = nc.variables['lat'][:]
            lon = nc.variables['lon'][:]
            data = nc.variables['chlor_a'][:]  # Adjust for your variable
            
            # Crop data
            lat_inds = np.where((lat >= lat_min_crop) & (lat <= lat_max_crop))[0]
            lon_inds = np.where((lon >= lon_min_crop) & (lon <= lon_max_crop))[0]
            cropped_data = data[:, lat_inds, :][:, :, lon_inds]
            cropped_data = np.where(cropped_data.mask, np.nan, cropped_data)
            
            if land_mask_accumulator is None:
                land_mask_accumulator = np.isnan(cropped_data)
            else:
                land_mask_accumulator &= np.isnan(cropped_data)

# Final land mask (True where land is detected)
land_mask = land_mask_accumulator.all(axis=0)


GS20180427_CHL_NW_day.nc
GS20180807_CHL_NW_day.nc
GS20181117_CHL_NW_day.nc
GS20190228_CHL_NW_day.nc
GS20190610_CHL_NW_day.nc
GS20190920_CHL_NW_day.nc
GS20191230_CHL_NW_day.nc
GS20200411_CHL_NW_day.nc
GS20200721_CHL_NW_day.nc
GS20201101_CHL_NW_day.nc
GS20210211_CHL_NW_day.nc
GS20210523_CHL_NW_day.nc
GS20210903_CHL_NW_day.nc
GS20211213_CHL_NW_day.nc
GS20220325_CHL_NW_day.nc
GS20220705_CHL_NW_day.nc
GS20221015_CHL_NW_day.nc
GS20230125_CHL_NW_day.nc
GS20230507_CHL_NW_day.nc
GS20230817_CHL_NW_day.nc
GS20231130_CHL_NW_day.nc


In [ ]:
good_data = []
print(good_data)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plot, colors
# Define your color list in hex
hex_colors_chl = [
    '#3500a8', '#0800ba', '#003fd6',
    '#00aca9', '#77f800', '#ff8800', 
    '#b30000', '#920000', '#880000'
]
rgb_colors = [colors.hex2color(color) for color in hex_colors_chl]

import numpy as np
from matplotlib import pyplot as plot, colors
from mpl_toolkits import basemap
from netCDF4 import Dataset, num2date
from matplotlib.colors import LinearSegmentedColormap


In [ ]:

missing_data_info = {}
total_data_info = {}
days_in_month_info = {}
data_dir = '../results/daily/gcomc/chla'
days_in_month = 0
current_month = 0
count  = 0
# Second pass to calculate missing data excluding land
for file_name in os.listdir(data_dir):
    if not file_name.endswith('.nc'):
       continue
    count += 1
    if count % 100 == 0:
        print(file_name)

    if file_name.endswith('.nc'):
        year_month = file_name[2:10]

        with Dataset(os.path.join(data_dir, file_name), 'r') as nc:
            lat = nc.variables['lat'][:]
            lon = nc.variables['lon'][:]
            data = nc.variables['chlor_a'][:]  # Adjust for your variable
            # Crop data again
            lat_inds = np.where((lat >= lat_min_crop) & (lat <= lat_max_crop))[0]
            lon_inds = np.where((lon >= lon_min_crop) & (lon <= lon_max_crop))[0]
            cropped_data = data[:, lat_inds, :][:, :, lon_inds]
            # print(cropped_data.min(), cropped_data.max())
            cropped_data = np.where(cropped_data.mask, np.nan, cropped_data)
            
            # Exclude land pixels from the missing data calculation
            valid_data_mask = ~np.isnan(cropped_data) | land_mask[np.newaxis, :, :]
            missing_data = np.isnan(cropped_data) & ~land_mask[np.newaxis, :, :]
            # print(missing_data.sum())
            # missing_data_info[file_name] = missing_data.sum()
            # total_data_info[file_name] = valid_data_mask.sum()
            # print(file_name, year_month)
            # print(missing_data.sum())
            # print(valid_data_mask.sum())
            # print(missing_data.sum()/ valid_data_mask.sum() * 100)
            # check if missind_data_info has the year_month key
            # print(year_month)
            month  = int(year_month[4:6])
            if month != current_month:
                days_in_month_info[year_month[:6]] = 0
                days_in_month = 0
                current_month = month
            if year_month not in missing_data_info:
                missing_data_info[year_month] = {'missing_pixels': 0, 'total_pixels': 0, 'days_counted': 0}
            
            avg_missing_percentage = (missing_data.sum() / valid_data_mask.sum()) * 100
            print(avg_missing_percentage)
            if avg_missing_percentage < 75:
                days_in_month += 1
            if avg_missing_percentage < 30:
                days_in_month_info[year_month[:6]] = days_in_month_info[year_month[:6]] + 1
            else:
                continue
                # save the image
                # print("days in month", days_in_month, days_in_month_info)
            # missing_data_info[year_month]['missing_pixels'] += missing_data.sum()
            # missing_data_info[year_month]['total_pixels'] += valid_data_mask.sum()
            # missing_data_info[year_month]['days_counted'] += 1

            sds = np.ma.squeeze(data) # remove singleton dimensions
            label = nc["chlor_a"].units.replace('^-3', '$^{-3}$')
            lat = nc['lat'][:]
            lon = nc['lon'][:]
        
            # Find indices for cropping
            lat_inds = np.where((lat >= lat_min_crop) & (lat <= lat_max_crop))[0]
            lon_inds = np.where((lon >= lon_min_crop) & (lon <= lon_max_crop))[0]
            print(sds)
            # Crop data
                # Now add a check to ensure the indices are within the bounds of the array
            if lat_inds.size > 0 and lon_inds.size > 0:
                # Adjust the slicing based on the actual dimensions of sds
                sds_cropped = sds[lat_inds, :][:, lon_inds]  # This is the updated line for a 2D array
                lat_cropped = lat[lat_inds]
                lon_cropped = lon[lon_inds]
            else:
                print("No data within specified crop bounds.")
                
    
    print("vis " ,avg_missing_percentage, filename)
    # Visualisation with basemap
    if len(lon_cropped.shape) == 1:
        lon_cropped, lat_cropped = np.meshgrid(lon_cropped, lat_cropped)
    lon_0, lat_0 = (lon_cropped.min() + lon_cropped.max()), (lat_cropped.min() + lat_cropped.max()) / 2
    m = basemap.Basemap(llcrnrlon=lon_cropped.min(), llcrnrlat=lat_cropped.min(), 
                        urcrnrlon=lon_cropped.max(), urcrnrlat=lat_cropped.max(), resolution='i', 
                        lon_0=lon_0, lat_0=lat_0, projection='merc')

    sds_cropped  = cropped_data
    sds_cropped_log = np.ma.masked_less_equal(sds_cropped, 0)  # Mask non-positive values
    sds_cropped_log = np.ma.log10(sds_cropped_log)  # Apply log10 to the data

    # Adjust figsize to change the aspect ratio
    fig, ax = plot.subplots(figsize=(7, 6))  # Adjust the width and height to better suit your data aspect ratio

    # figure bounds
    extent = [lon_cropped.min(), lon_cropped.max(), lat_cropped.min(), lat_cropped.max()]

    # Land mask
    mask = np.where(~sds_cropped_log.mask, np.nan, 0)
    ax.imshow(mask, cmap='gray', vmin=-2, vmax=0, extent=extent)

    # Create a colormap object
    custom_colormap = LinearSegmentedColormap.from_list('custom', rgb_colors)

    print(sds_cropped_log.max(), sds_cropped_log.min())
    # We no longer use LogNorm here since we've manually applied log10
    # ims = ax.imshow(sds_cropped_log, cmap='jet', extent=extent)
    ims = ax.imshow(sds_cropped_log,vmin=cmin, vmax=cmax,  cmap=custom_colormap, extent=extent)

    # # Figure labels
    ax.set_xlabel('Longitude [$^\mathregular{o}$E]', fontsize="12")
    ax.set_ylabel('Latitude [$^\mathregular{o}$N]', fontsize="12")
    ax.set_yticks(range(int(np.ceil(lat_cropped.min())), int(np.ceil(lat_cropped.max())) + 1, 1))
    # ax.set_title(time[0].strftime('%b %Y'))

    # # Colourbar
    cbar = fig.colorbar(ims, ax=ax, orientation='vertical', fraction=0.0324, pad=0.025,  aspect=15)
    # cbar.set_label('log10(Chl-a) Conc. mg/m^3', fontsize="12")

    ticks = [ims.get_clim()[0],0, ims.get_clim()[1]] # This gets the color limit range
    cbar.set_ticks(ticks)
    cbar.set_ticklabels([f'{ticks[0]:.2f}', f'{ticks[1]:.2f}', f'{ticks[2]:.2f}']) # Format as desired

    # plot.savefig("test.png", dpi=300)
    # plot.savefig(f'./results/daily/Modis/chla/pics/{file.split("/")[-1].split(".")[0]}.png', dpi=300)
    plot.show()
  